In [1]:
from budget_builder.models import Expense, Category
import sqlalchemy as sa
import polars as pl
from sqlalchemy.orm import Session
import altair as alt
import pandas as pd

In [2]:
engine = sa.create_engine("sqlite:///../test.db")

In [3]:
expense_table = Expense.__table__
category_table = Category.__table__
sql = sa.select(expense_table.c.date, expense_table.c.amount, expense_table.c.description, category_table.c.name, category_table.c.is_fixed_cost).join(category_table).order_by(expense_table.c.date)

In [4]:
with engine.connect() as conn:
    df = pd.read_sql(sql, conn).assign(date=lambda x: pd.to_datetime(x['date'])).set_index('date').query('name != "Transfer"')

In [16]:
df[df.name == 'Games'].sort_values('amount')

,amount,description,name,is_fixed_cost
date,,,,
2022-12-29,-1046.46,STEAM PURCHASE SEA,Games,False
2022-06-29,-914.42,STEAM PURCHASE SEA,Games,False
2022-02-10,-753.95,VANLOESE BOG & IDE,Games,False
2022-08-02,-752.09,Xsolla _MTGARENA L,Games,False
2022-11-17,-751.38,Xsolla MTGARENA,Games,False
...,...,...,...,...
2022-12-13,-20.00,FUNTASIA LEGELAND AP,Games,False
2022-12-05,-20.00,MobilePay CPSI festa,Games,False
2022-09-14,-16.92,"GBP 1,95 HYDE PARK CORNER LOD",Games,False


In [7]:
per_category = df.groupby([pd.Grouper(freq='m'), 'is_fixed_cost', 'name']).agg({'amount': 'sum'}).unstack('name').assign(total=lambda x: x.sum(axis=1))
totals = per_category.total.mul(-1).reset_index()

In [13]:
per_category.loc[pd.IndexSlice[:, False], pd.IndexSlice['amount', 'Games']]

date        is_fixed_cost
2021-12-31  False            -854.00
2022-01-31  False                NaN
2022-02-28  False           -1721.69
2022-03-31  False            -828.57
2022-04-30  False           -1433.94
2022-05-31  False           -1572.85
2022-06-30  False           -1174.29
2022-07-31  False           -1318.47
2022-08-31  False           -2991.52
2022-09-30  False           -1529.31
2022-10-31  False           -1114.00
2022-11-30  False           -1909.47
2022-12-31  False           -3564.19
2023-01-31  False            -741.43
2023-02-28  False           -2088.35
2023-03-31  False            -525.99
2023-04-30  False            -685.00
2023-05-31  False            -219.95
2023-06-30  False                NaN
Name: (amount, Games), dtype: float64

In [9]:
per_category.swaplevel().loc[pd.IndexSlice[False]].dropna(how='all', axis=1)['amount'].fillna(0).round(0).mul(-1).describe()

name,Babysitter,Cleaning,Dentist,Games,Household Items,Lunch,Maintenance,Restaurant,School,Supermarket,Training,Transportation,Travel,Unknown,Yearly Card
count,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000,19.000000
mean,773.684211,2001.526316,402.473684,1277.473684,6190.210526,338.263158,2846.894737,6616.578947,856.315789,5866.473684,26.684211,986.315789,1255.473684,3860.315789,244.842105
std,584.926021,1113.874787,1216.031084,930.993756,5569.293218,211.069984,7256.695260,3367.995618,2059.876508,2662.440722,85.550539,787.082803,1799.876273,6385.857987,483.712755
min,-0.000000,600.000000,-0.000000,-0.000000,516.000000,-0.000000,-0.000000,302.000000,-0.000000,365.000000,-0.000000,30.000000,-0.000000,-0.000000,-591.000000
25%,330.000000,1393.500000,-0.000000,713.000000,2109.000000,212.000000,-0.000000,4866.000000,-0.000000,5140.000000,-0.000000,427.000000,-0.000000,75.000000,-0.000000
50%,660.000000,2062.000000,0.000000,1174.000000,4383.000000,411.000000,0.000000,6036.000000,0.000000,6523.000000,0.000000,798.000000,0.000000,776.000000,0.000000
75%,1127.500000,2357.500000,247.000000,1647.500000,7977.000000,464.000000,1078.000000,8858.500000,-0.000000,7122.000000,-0.000000,1417.000000,2625.000000,5786.500000,418.000000
max,1860.000000,5565.000000,5309.000000,3564.000000,19447.000000,619.000000,29062.000000,12600.000000,7700.000000,10404.000000,345.000000,3123.000000,5245.000000,24008.000000,1578.000000


In [12]:
alt.Chart(totals).mark_bar().encode(x='date', y='total', column='is_fixed_cost', tooltip='total').interactive()

alt.Chart(...)

In [10]:
alt.Chart(yearly_cost).mark_bar().encode(x='date', y='amount', facet='name', tooltip='amount')

NameError: name 'yearly_cost' is not defined